## used to be inspired from: 
https://medium.com/@anthony.galtier/fine-tuning-bert-for-a-regression-task-is-a-description-enough-to-predict-a-propertys-list-price-cf97cd7cb98a

Since we changed to classification it is freestyle now :)

https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import re
import string
import os
from PIL import Image

In [ ]:
X_train = pickle.load(open("drive/MyDrive/ABA project/X_train.p", "rb"))
X_val =  pickle.load(open("drive/MyDrive/ABA project/X_val.p", "rb"))
X_test =  pickle.load(open("drive/MyDrive/ABA project/X_test.p", "rb"))
y_train = pickle.load(open("drive/MyDrive/ABA project/y_train.p", "rb"))
y_val = pickle.load(open("drive/MyDrive/ABA project/y_val.p", "rb"))
y_test = pickle.load(open("drive/MyDrive/ABA project/y_test.p", "rb"))

In [ ]:
X_train_source = np.array(pd.get_dummies(X_train['source_name']))
X_val_source = np.array(pd.get_dummies(X_val['source_name']))
X_test_source = np.array(pd.get_dummies(X_test['source_name']))

X_train_image = np.array(X_train['filepath'])
X_val_image = np.array(X_val['filepath'])
X_test_image = np.array(X_test['filepath'])

X_train_text = list(X_train['title'] + X_train['description'])
X_val_text = list(X_val['title'] + X_val['description'])
X_test_text = list(X_test['title'] + X_test['description'])
y_train = list(np.log1p(y_train['engagement']))
y_val = list(np.log1p(y_val['engagement']))
y_test = list(np.log1p(y_test['engagement']))

In [ ]:
#X_train_image = ["9645.jpg","9646.jpg"]
#X_train_text = X_train_text[0:2]
#X_train_source = X_train_source[0:2]
#y_train = y_train[0:2]
#X_train_image

## Super model
Trying to combine text, image and source_name to form a SuperModel!

In [ ]:
!pip3 install transformers==4.8.2
!pip install wandb

     |████████████████████████████████| 2.5 MB 9.9 MB/s 
     |████████████████████████████████| 880 kB 45.0 MB/s 
     |████████████████████████████████| 3.3 MB 42.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=58b5bdb6d48381e96edcdc30e1dd105c36c474eb8481d89a8d0dfd140a1df40d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
     |████████████████████████████████| 1.8 MB 7.4 MB/s 
     |████████████████████████████████| 144 kB 50.8 MB/s 
     |████████████████████████████████| 181 kB 54.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=8007abf86be083ed0e370007183a29a8c0065026c40469d24f07af8339941c0a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AutoConfig
from transformers import EarlyStoppingCallback
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from torchvision import transforms
import torch.nn.functional as F


In [ ]:
#Custom model
class CNNModel(nn.Module):
    def __init__(self, drop_rate=0.2):
        super().__init__()

        D_in_image = 10
        D_out = 1

        #cnn based on images
        input_channels = 3
        self.conv1 = nn.Conv2d(in_channels=input_channels,
                               out_channels=16,
                               kernel_size=5,)
        self.conv2 = nn.Conv2d(in_channels=16, 
                               out_channels=32,
                               kernel_size=3)
        #self.conv2_drop = nn.Dropout2d()
        self.l_1_image = nn.Linear(in_features = 32, out_features = 10)
        


        #dense layer
        self.dropout = nn.Dropout(drop_rate)
        self.regressor = nn.Linear(D_in_Bert+D_in_source+ D_in_image, D_out) 

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        source=None,
        image=None,
    ):

        #image
        x = image
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = F.max_pool2d(x,(28,28))
        x = x.squeeze()
        x = F.relu(self.l_1_image(x))

        logits = self.regressor(x)#(pooled_output)

        loss = None
        if labels is not None:
            #  We are doing regression
            loss_fct = MSELoss()
            loss = loss_fct(logits.view(-1), labels.view(-1))
            

        return (loss, logits)


In [ ]:
model = CNNModel(drop_rate=0.2)

In [ ]:
#Image: Create method that transform image from .jpeg to tensor
#We need to resize and normalize picture
transform = transforms.Compose([
  transforms.Resize((120,120)),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
  ])

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, source, image, image_transform, labels=None):
        self.encodings = encodings
        self.labels = labels
        self.source = source

        self.image_transform = image_transform
        self.image_path = image

    def __getitem__(self, idx):
        item = {}
        img_loc = self.image_path[idx]
        image = Image.open(img_loc).convert("RGB")
        item["image"] = self.image_transform(image) #transform to tensor, rescale and normalizeW
        if self.labels:
            item["labels"] = torch.squeeze(torch.tensor(self.labels[idx]))
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_image, transform, y_train)
val_dataset = Dataset(X_val_image, transform, y_val)

In [ ]:
# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters

# Define Trainer
args = TrainingArguments(
    output_dir="image",
    evaluation_strategy="epoch",
    eval_steps=1,
    logging_steps = 1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    load_best_model_at_end=True,
    report_to="wandb",
    learning_rate = 0.001,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics_reg,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

PyTorch: setting up devices


In [ ]:
# Train pre-trained model
trainer.train()

***** Running training *****
  Num examples = 6278
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11790
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Rmse,Mse,R2,Mae
1,5.752400,7.412351,2.722563,7.412351,-0.023498,2.275134
2,2.979600,7.215787,2.686222,7.215787,0.003643,2.240338
3,2.323400,6.786705,2.605130,6.786705,0.062891,2.153294
4,9.362900,6.629194,2.574722,6.629194,0.084640,2.152106
5,2.804700,6.705364,2.589472,6.705364,0.074123,2.135092
6,4.593700,6.603457,2.569719,6.603457,0.088194,2.124525
7,3.559300,6.626361,2.574172,6.626361,0.085031,2.137789
8,5.002800,6.595428,2.568156,6.595428,0.089303,2.120844
9,3.331700,6.638323,2.576494,6.638323,0.083380,2.121290
10,11.321500,7.008771,2.647408,7.008771,0.032228,2.242180


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
***** Running Evaluation *****
  Num examples = 698
  Batch size = 16
Saving model checkpoint to image/checkpoint-393
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
***** Running Evaluation *****
  Num examples = 698
  Batch size = 16
Saving model checkpoint to image/checkpoint-786
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images w

TrainOutput(global_step=7074, training_loss=6.6870990285756795, metrics={'train_runtime': 5664.0779, 'train_samples_per_second': 33.252, 'train_steps_per_second': 2.082, 'total_flos': 0.0, 'train_loss': 6.6870990285756795, 'epoch': 18.0})

In [ ]:
# ----- 3. Predict -----#

# Create torch dataset
test_dataset = Dataset(X_test_image, transform, y_test)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
y_pred, _, _ = test_trainer.predict(test_dataset)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 1744
  Batch size = 8


In [ ]:
evaluate(y_pred, y_test)

{'mae': 2.105758069301436,
 'mse': 6.476038443584958,
 'r2': 0.07962411143438064,
 'rmse': 2.5448061701404603}

In [ ]:
torch.save(model,"drive/MyDrive/ABA project/NN_models/image")